# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])
nltk.download('averaged_perceptron_tagger')

import re
import numpy as np
import pandas as pd
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///ble_etl_pipeline.db')
df = pd.read_sql_table('ble_etl_pipeline',engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#Generate descriptive statistics - get a sense of what the data looks like
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,...,26207.000000,26207.00000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000,26207.000000
mean,15142.682375,0.767200,0.170947,0.004541,0.415080,0.079635,0.050139,0.027626,0.017972,0.032816,...,0.011791,0.04392,0.278628,0.082344,0.093410,0.010760,0.093601,0.020224,0.052505,0.193841
std,8780.194298,0.422624,0.376470,0.067233,0.492745,0.270733,0.218236,0.163902,0.132853,0.178157,...,0.107945,0.20492,0.448333,0.274894,0.291012,0.103175,0.291278,0.140767,0.223047,0.395314
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7408.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15568.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22805.500000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30111.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [5]:
#Define feature and target variables X and Y
X = df["message"]
Y = df.drop(['id','message','original','genre'], axis=1) #Y = df.iloc[:,4:]

In [6]:
#check to see if there are missing values in X and Y
X.isnull().sum()

0

In [7]:
Y.isnull().sum()

related                   0
request                   0
offer                     0
aid_related               0
medical_help              0
medical_products          0
search_and_rescue         0
security                  0
military                  0
child_alone               0
water                     0
food                      0
shelter                   0
clothing                  0
money                     0
missing_people            0
refugees                  0
death                     0
other_aid                 0
infrastructure_related    0
transport                 0
buildings                 0
electricity               0
tools                     0
hospitals                 0
shops                     0
aid_centers               0
other_infrastructure      0
weather_related           0
floods                    0
storm                     0
fire                      0
earthquake                0
cold                      0
other_weather             0
direct_report       

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    """
    input: message that needs to be tokenized
    
    output: message that is normalized, lemmatized, with leading/trailing white spaces removed 
    """
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [11]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
#compute evaluation metrics on training set
y_pred_train = pipeline.predict(X_train)

#compute evaluation metrics on test set
y_pred_test = pipeline.predict(X_test)

In [13]:
target_names = Y.columns.values
target_names

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'], dtype=object)

In [14]:
#check the dimensions of the dataframes
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((19655,), (6552,), (19655, 36), (6552, 36))

In [15]:
#check which object type is y_pred_test
type(y_pred_test)

numpy.ndarray

In [16]:
#change y_pred_test to dataframe
y_pred_test = pd.DataFrame(y_pred_test)
type(y_pred_test) #confirm it has been converted to a dataframe

pandas.core.frame.DataFrame

In [17]:
type(y_test)

pandas.core.frame.DataFrame

In [18]:
#print classification report for test set
print(classification_report(y_test.values, y_pred_test, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.77      0.93      0.84      5017
               request       0.40      0.06      0.11      1158
                 offer       0.00      0.00      0.00        30
           aid_related       0.45      0.20      0.28      2726
          medical_help       0.12      0.00      0.00       528
      medical_products       0.06      0.00      0.01       325
     search_and_rescue       0.00      0.00      0.00       174
              security       0.00      0.00      0.00       115
              military       0.00      0.00      0.00       219
           child_alone       0.00      0.00      0.00         0
                 water       0.12      0.01      0.01       441
                  food       0.08      0.00      0.01       748
               shelter       0.04      0.00      0.00       604
              clothing       0.50      0.01      0.02       100
                 money       0.00      

In [19]:
#check which object type is y_pred_train
type(y_pred_train)

numpy.ndarray

In [20]:
#change y_pred_train to dataframe
y_pred_train = pd.DataFrame(y_pred_train)
type(y_pred_train) #confirm it has been converted to a dataframe

pandas.core.frame.DataFrame

In [21]:
#print classification report for training set
print(classification_report(y_train.values, y_pred_train, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.98      1.00      0.99     15089
               request       1.00      0.86      0.92      3322
                 offer       1.00      0.72      0.84        89
           aid_related       0.99      0.94      0.97      8152
          medical_help       1.00      0.74      0.85      1559
      medical_products       0.99      0.72      0.84       989
     search_and_rescue       1.00      0.73      0.85       550
              security       1.00      0.69      0.82       356
              military       1.00      0.73      0.84       641
           child_alone       0.00      0.00      0.00         0
                 water       0.99      0.75      0.85      1233
                  food       1.00      0.80      0.89      2182
               shelter       1.00      0.76      0.86      1715
              clothing       1.00      0.73      0.84       306
                 money       1.00      

In [22]:
#check accuracy of the model for the test set
model_accuracy_test  = (y_pred_test.values == y_test).mean().mean()
print('The model accuracy for the test set is {:.5f}'.format(model_accuracy_test))

The model accuracy for the test set is 0.92396


### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
#checkout the pipeline parameters
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f96283ccf28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [24]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [25]:
parameters =  {
        'clf__estimator__n_estimators': [10],
        'clf__estimator__min_samples_split': [2, 4],
        
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

In [27]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
#compute evaluation metrics on training set for the grid search tunning
y_pred_train_cv = cv.predict(X_train)

#compute evaluation metrics on test set for the grid search tunning
y_pred_test_cv = cv.predict(X_test)

In [29]:
#check which object type is y_pred_test
type(y_pred_test_cv)

numpy.ndarray

In [30]:
#change y_pred_test to dataframe
y_pred_test_cv = pd.DataFrame(y_pred_test)
type(y_pred_test_cv) #confirm it has been converted to a dataframe

pandas.core.frame.DataFrame

In [31]:
type(y_test)

pandas.core.frame.DataFrame

In [32]:
model_accuracy_test_cv  = (y_pred_test_cv.values == y_test).mean().mean()
print('The model accuracy for the test set with grid search is {:.5f}'.format(model_accuracy_test))

The model accuracy for the test set with grid search is 0.92396


In [33]:
#print classification report for test set for the grid search tunning
print(classification_report(y_test.values, y_pred_test_cv, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.77      0.93      0.84      5017
               request       0.40      0.06      0.11      1158
                 offer       0.00      0.00      0.00        30
           aid_related       0.45      0.20      0.28      2726
          medical_help       0.12      0.00      0.00       528
      medical_products       0.06      0.00      0.01       325
     search_and_rescue       0.00      0.00      0.00       174
              security       0.00      0.00      0.00       115
              military       0.00      0.00      0.00       219
           child_alone       0.00      0.00      0.00         0
                 water       0.12      0.01      0.01       441
                  food       0.08      0.00      0.01       748
               shelter       0.04      0.00      0.00       604
              clothing       0.50      0.01      0.02       100
                 money       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [34]:
#create new pipeline for AdaBoostClassifier
pipeline_ada = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
                ])

In [35]:
#get parameters for AdaBoostClassifier
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f96283ccf28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=42),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=

In [36]:
#create new parameter
parameters_ada =  {
        'tfidf__use_idf': (True,False),
        'clf__estimator__n_estimators': [50,100],
        'clf__estimator__learning_rate':[0.5],
        'clf__estimator__random_state': [42]
         }

cv_ada = GridSearchCV(pipeline_ada, param_grid=parameters_ada, cv=2, verbose=3)
#cv=2 and verbose=3 helps pipeline to fit faster with realtime training info

In [37]:
cv_ada.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, clf__estimator__random_state=42, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, clf__estimator__random_state=42, tfidf__use_idf=True, score=0.17897842897842897, total= 1.4min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, clf__estimator__random_state=42, tfidf__use_idf=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, clf__estimator__random_state=42, tfidf__use_idf=True, score=0.1827617787727689, total= 1.4min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, clf__estimator__random_state=42, tfidf__use_idf=False 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.0min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, clf__estimator__random_state=42, tfidf__use_idf=False, score=0.18284493284493283, total= 1.2min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, clf__estimator__random_state=42, tfidf__use_idf=False 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=50, clf__estimator__random_state=42, tfidf__use_idf=False, score=0.18550931108171365, total= 1.2min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, clf__estimator__random_state=42, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, clf__estimator__random_state=42, tfidf__use_idf=True, score=0.16432641432641432, total= 2.5min
[CV] clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, clf__estimator__random_state=42, tfidf__use_idf=True 
[CV]  clf__estimator__learning_rate=0.5, clf__estimator__n_estimators=100, clf__estimator__random_state=42

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 16.1min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...timator=None,
          learning_rate=1.0, n_estimators=50, random_state=42),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__learning_rate': [0.5], 'clf__estimator__random_state': [42]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [40]:
#compute evaluation metrics on test set for the grid search tunning for 
y_pred_test_cv_ada = cv_ada.predict(X_test)

In [41]:
accuracy_cv_ada = (y_pred_test_cv_ada == y_test.values).mean().mean()
accuracy_cv_ada

0.92651098901098905

### 9. Export your model as a pickle file

In [42]:
filename = 'disaster_response_model.sav'
pickle.dump(pipeline_ada, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.